In [124]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv', index_col=[0], header=[0,1])

df.index /= 200
df.rename(columns={'q' : })

accel                           gyro                            mag  \
              x         y          z         x         y         z          x   
0.000 -1.192331  2.236219  -9.483578 -0.013341 -0.036044 -0.018854  44.203568   
0.005 -1.192331  2.236219  -9.483578 -0.013341 -0.036044 -0.018854  44.203568   
0.010 -1.192331  2.236219  -9.483578 -0.013341 -0.036044 -0.018854  44.203568   
0.015 -1.192331  2.236219  -9.483578 -0.013341 -0.036044 -0.018854  44.203568   
0.020 -1.192331  2.236219  -9.483578 -0.013341 -0.036044 -0.018854  44.203568   
...         ...       ...        ...       ...       ...       ...        ...   
0.005 -0.574617  1.740611 -10.244946  0.007166 -0.111945  0.067966  41.576201   
0.010 -0.574617  1.740611 -10.244946  0.007166 -0.111945  0.067966  41.576201   
0.015 -0.574617  1.740611 -10.244946  0.007166 -0.111945  0.067966  41.576201   
0.020 -0.574617  1.740611 -10.244946  0.007166 -0.111945  0.067966  41.576201   
0.025 -0.574617  1.740611 -10.244946  0.007166 -0.111945  0.067966  41.576201   

                                                                        (Q, Q)  
               y          z                                Unnamed: 10_level_1  
0.000 -11.340037  45.217584  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.005 -11.340037  45.217584  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.010 -11.340037  45.217584  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.015 -11.340037  45.217584  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.020 -11.340037  45.217584  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
...          ...        ...                                                ...  
0.005   2.548464  55.824087  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.010   2.548464  55.824087  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.015   2.548464  55.824087  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.020   2.548464  55.824087  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  
0.025   2.548464  55.824087  [ 0.00294132 -0.60259825 -0.79547429 -0.06393232]  

[4260 rows x 10 columns]

In [102]:
import imufusion

sample_rate = 200

offset = imufusion.Offset(sample_rate)
ahrs = imufusion.Ahrs()

ahrs.settings = imufusion.Settings(imufusion.CONVENTION_NED,
                                   0.5,  # gain
                                   500,  # gyroscope range
                                   10,  # acceleration rejection
                                   0,  # magnetic rejection
                                   5 * sample_rate)  # rejection timeout = 5 seconds

def update(x):
    ahrs.update(x['gyro'].to_numpy(), x['accel'].to_numpy(), x['mag'].to_numpy(), 0.005)

    euler = ahrs.quaternion.to_euler()
    Q = ahrs.quaternion.wxyz

    ahrs_internal_states = ahrs.internal_states
    
    acceleration = ahrs.earth_acceleration  # convert g to m/s/
    ans = {}
    ans.update({'x': acceleration[1], 'y':acceleration[1], 'z':acceleration[2]})
    ans.update({'yaw': euler[0], 'pitch' : euler[1], 'roll' : euler[2]})
    ans.update({'Q_T' : Q})
    # ans.update({"accel_err" : ahrs.internal_states.acceleration_error})
    # ans.update({"accel_igr" : ahrs.internal_states.accelerometer_ignored})
    # ans.update({"accel_rec" : ahrs.internal_states.acceleration_recovery_trigger})

    # ans.update({"mag_err" : ahrs.internal_states.magnetic_error})
    # ans.update({"mag_igr" : ahrs.internal_states.magnetometer_ignored})
    # ans.update({"mag_rec" : ahrs.internal_states.magnetic_recovery_trigger})
    
    # ans.update({"ang_rrec" : ahrs.flags.angular_rate_recovery})
    # ans.update({"accel_rrec" : ahrs.flags.acceleration_recovery})
    # ans.update({"mag_rrec" : ahrs.flags.magnetic_recovery})
    return ans

hf = df.apply(update, axis=1)

hf = pd.DataFrame(list(hf))

hf



,x,y,z,yaw,pitch,roll,Q_T
0,2.113579,2.113579,-8.515456,-0.653778,-0.343531,0.712614,"[0.0, 0.0, 8e-45, 2.989e-42]"
1,1.998336,1.998336,-8.544278,-1.274752,-0.670443,1.366612,"[1.9983356, 0.0, -0.0059825643, 0.011859634]"
2,1.890064,1.890064,-8.570340,-1.864572,-0.981444,1.965081,"[1.890064, 0.0, -0.00884121, 0.017005496]"
3,1.788344,1.788344,-8.593899,-2.424806,-1.277229,2.511066,"[1.7883437, 0.0, -0.011604116, 0.021669392]"
4,1.692767,1.692767,-8.615189,-2.956948,-1.558477,3.007551,"[1.6927674, 0.0, -0.014267684, 0.025880886]"
...,...,...,...,...,...,...,...
4255,-0.821701,-0.821701,-9.369071,-14.542367,-3.814600,-30.776485,"[-0.82170117, 0.0, 0.0017353784, -0.2671399]"
4256,-0.819811,-0.819811,-9.369265,-14.530120,-3.813469,-30.758663,"[-0.8198111, 0.0, 0.0016959494, -0.26699045]"
4257,-0.817925,-0.817925,-9.369454,-14.517902,-3.812341,-30.740839,"[-0.81792545, 0.0, 0.0016566068, -0.26684102]"
4258,-0.816044,-0.816044,-9.369640,-14.505714,-3.811215,-30.723015,"[-0.81604445, 0.0, 0.0016173503, -0.2666916]"


In [114]:
import itertools
import matplotlib.pyplot as plt
from PDR.pdr import PDR

dt = 1 / 200

kf = hf[['x', 'y', 'z']].copy()

kf['qtrm'] = df['q'].apply(PDR.quat_to_rot_mat)
# df = df.apply(lambda x: print(x['qtrm'][0], x['accel'].to_numpy()), axis=1)
kf = kf.apply(lambda x: x['qtrm'][0] @ x.to_numpy(), axis=1).to_list()

# velocity
kf.columns = tuple(itertools.product(['accel'], ['x', 'y', 'z']))
sf = kf.shift(1).fillna(0).apply(lambda x: x * dt)
sf.columns = (('vel', 'x'), ('vel', 'y'), ('vel', 'z'))
sf = sf.cumsum()
kf = pd.concat([kf, sf], axis=1)
# position

sf = pd.DataFrame(0, index=kf.index, columns=(('pos', 'x'), ('pos', 'y'), ('pos', 'z')))
kf = pd.concat([kf, sf], axis=1)
kf['pos'] = kf['vel'].shift(1).fillna(0) * dt + kf['accel'].shift(1) * (dt ** 2) / 2
kf['pos'] = kf['pos'].cumsum()

# plot

kf = kf.iloc[:20]
fig, ax = plt.subplots(1,3, figsize=(15, 10), tight_layout=True)
ax.flatten ()

ax[0].plot(kf['accel']['x'], kf['accel']['y'], marker='>',label='accel')
ax[0].set_title('acceleration')
ax[0].axis('equal')
ax[0].grid()

ax[1].plot(kf['vel']['x'], kf['vel']['y'], marker='>', label='vel')
ax[1].set_title('velocity')
ax[1].axis('equal')
ax[1].grid()

ax[2].plot(kf['pos']['x'], kf['pos']['y'], marker='>', label='pos')
ax[2].set_title('position')
ax[2].axis('equal')
ax[2].grid()

plt.show()

TypeError: can't multiply sequence by non-int of type 'str'

In [115]:
kf

,x,y,z
0,2.113579,2.113579,-8.515456
1,1.998336,1.998336,-8.544278
2,1.890064,1.890064,-8.570340
3,1.788344,1.788344,-8.593899
4,1.692767,1.692767,-8.615189
...,...,...,...
4255,-0.821701,-0.821701,-9.369071
4256,-0.819811,-0.819811,-9.369265
4257,-0.817925,-0.817925,-9.369454
4258,-0.816044,-0.816044,-9.369640


In [121]:
# PDR.quat_to_rot_mat(df)
df['q']

,Unnamed: 10_level_1
0.000,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.005,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.010,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.015,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.020,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
...,...
0.005,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.010,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.015,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
0.020,[ 0.00294132 -0.60259825 -0.79547429 -0.06393232]
